<a id='up'></a>

## Содержание jupyter notebook (ссылки):
1. Подключение библиотек и функций для получения вейвлет-спектра
2. [Генерация 40000 синтетических сигналов](#gen)
3. [Получение информации об областях значимости](#info)
4. Нейронные сети:
&emsp;[Перцептрон](#network1)
&emsp;[Нейронная сеть с двумя входами](#network2)
&emsp;[Сверточная нейронная сеть](#network3)

In [1]:
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec

import numpy as np
import pandas as pd
import scipy
import scipy.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline
import random
from astropy.io import fits

from sklearn.model_selection import train_test_split

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils.np_utils import to_categorical

from scipy.optimize import curve_fit

import emd

from shapely.geometry import Polygon

**В поле ниже длинный код функции для вейвлет-преобразования, взятой с GitHub**

In [2]:
# Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
# Python version of the code is written by Evgeniya Predybaylo in 2014
# edited by Michael von Papen (FZ Juelich, INM-6), 2018, to include
# analysis at arbitrary frequencies
#
#   This software may be used, copied, or redistributed as long as it is not
#   sold and this copyright notice is reproduced on each copy made. This
#   routine is provided as is without any express or implied warranties
#   whatsoever.
#
# Notice: Please acknowledge the use of the above software in any publications:
#            Wavelet software was provided by C. Torrence and G. Compo,
#      and is available at URL: http://paos.colorado.edu/research/wavelets/''.
#
# Reference: Torrence, C. and G. P. Compo, 1998: A Practical Guide to
#            Wavelet Analysis. <I>Bull. Amer. Meteor. Soc.</I>, 79, 61-78.
#
# Please send a copy of such publications to either C. Torrence or G. Compo:
#  Dr. Christopher Torrence               Dr. Gilbert P. Compo
#  Research Systems, Inc.                 Climate Diagnostics Center
#  4990 Pearl East Circle                 325 Broadway R/CDC1
#  Boulder, CO 80301, USA                 Boulder, CO 80305-3328, USA
#  E-mail: chris[AT]rsinc[DOT]com         E-mail: compo[AT]colorado[DOT]edu
#
# ----------------------------------------------------------------------------


# # WAVELET  1D Wavelet transform with optional significance testing
#   wave, period, scale, coi = wavelet(Y, dt, pad, dj, s0, J1, mother, param)
#
#   Computes the wavelet transform of the vector Y (length N),
#   with sampling rate DT.
#
#   By default, the Morlet wavelet (k0=6) is used.
#   The wavelet basis is normalized to have total energy=1 at all scales.
#
# INPUTS:
#
#    Y = the time series of length N.
#    DT = amount of time between each Y value, i.e. the sampling time.
#
# OUTPUTS:
#
#    WAVE is the WAVELET transform of Y. This is a complex array
#    of dimensions (N,J1+1). FLOAT(WAVE) gives the WAVELET amplitude,
#    ATAN(IMAGINARY(WAVE),FLOAT(WAVE) gives the WAVELET phase.
#    The WAVELET power spectrum is ABS(WAVE)**2.
#    Its units are sigma**2 (the time series variance).
#
# OPTIONAL INPUTS:
#
# *** Note *** if none of the optional variables is set up, then the program
#   uses default values of -1.
#
#    PAD = if set to 1 (default is 0), pad time series with zeroes to get
#         N up to the next higher power of 2. This prevents wraparound
#         from the end of the time series to the beginning, and also
#         speeds up the FFT's used to do the wavelet transform.
#         This will not eliminate all edge effects (see COI below).
#
#    DJ = the spacing between discrete scales. Default is 0.25.
#         A smaller # will give better scale resolution, but be slower to plot.
#
#    S0 = the smallest scale of the wavelet.  Default is 2*DT.
#
#    J1 = the # of scales minus one. Scales range from S0 up to S0*2**(J1*DJ),
#        to give a total of (J1+1) scales. Default is J1 = (LOG2(N DT/S0))/DJ.
#
#    MOTHER = the mother wavelet function.
#             The choices are 'MORLET', 'PAUL', or 'DOG'
#
#    PARAM = the mother wavelet parameter.
#            For 'MORLET' this is k0 (wavenumber), default is 6.
#            For 'PAUL' this is m (order), default is 4.
#            For 'DOG' this is m (m-th derivative), default is 2.
#
#
# OPTIONAL OUTPUTS:
#
#    PERIOD = the vector of "Fourier" periods (in time units) that corresponds
#           to the SCALEs.
#
#    SCALE = the vector of scale indices, given by S0*2**(j*DJ), j=0...J1
#            where J1+1 is the total # of scales.
#
#    COI = if specified, then return the Cone-of-Influence, which is a vector
#        of N points that contains the maximum period of useful information
#        at that particular time.
#        Periods greater than this are subject to edge effects.

import numpy as np

from scipy.optimize import fminbound
from scipy.special._ufuncs import gamma, gammainc

__author__ = 'Evgeniya Predybaylo, Michael von Papen'


def wavelet(Y, dt, pad=0, dj=-1, s0=-1, J1=-1, mother=-1, param=-1, freq=None):
    n1 = len(Y)

    if s0 == -1:
        s0 = 2 * dt
    if dj == -1:
        dj = 1. / 4.
    if J1 == -1:
        J1 = np.fix((np.log(n1 * dt / s0) / np.log(2)) / dj)
    if mother == -1:
        mother = 'MORLET'

    # construct time series to analyze, pad if necessary
    x = Y - np.mean(Y)
    if pad == 1:
        # power of 2 nearest to N
        base2 = np.fix(np.log(n1) / np.log(2) + 0.4999)
        nzeroes = (2 ** (base2 + 1) - n1).astype(np.int64)
        x = np.concatenate((x, np.zeros(nzeroes)))

    n = len(x)

    # construct wavenumber array used in transform [Eqn(5)]
    kplus = np.arange(1, int(n / 2) + 1)
    kplus = (kplus * 2 * np.pi / (n * dt))
    kminus = np.arange(1, int((n - 1) / 2) + 1)
    kminus = np.sort((-kminus * 2 * np.pi / (n * dt)))
    k = np.concatenate(([0.], kplus, kminus))

    # compute FFT of the (padded) time series
    f = np.fft.fft(x)  # [Eqn(3)]

    # construct SCALE array & empty PERIOD & WAVE arrays
    if mother.upper() == 'MORLET':
        if param == -1:
            param = 6.
        fourier_factor = 4 * np.pi / (param + np.sqrt(2 + param**2))
    elif mother.upper() == 'PAUL':
        if param == -1:
            param = 4.
        fourier_factor = 4 * np.pi / (2 * param + 1)
    elif mother.upper() == 'DOG':
        if param == -1:
            param = 2.
        fourier_factor = 2 * np.pi * np.sqrt(2. / (2 * param + 1))
    else:
        fourier_factor = np.nan

    if freq is None:
        j = np.arange(0, J1 + 1)
        scale = s0 * 2. ** (j * dj)
        freq = 1. / (fourier_factor * scale)
        period = 1. / freq
    else:
        scale = 1. / (fourier_factor * freq)
        period = 1. / freq
    # define the wavelet array
    wave = np.zeros(shape=(len(scale), n), dtype=complex)

    # loop through all scales and compute transform
    for a1 in range(0, len(scale)):
        daughter, fourier_factor, coi, _ = \
            wave_bases(mother, k, scale[a1], param)
        wave[a1, :] = np.fft.ifft(f * daughter)  # wavelet transform[Eqn(4)]

    # COI [Sec.3g]
    coi = coi * dt * np.concatenate((
        np.insert(np.arange(int((n1 + 1) / 2) - 1), [0], [1E-5]),
        np.insert(np.flipud(np.arange(0, int(n1 / 2) - 1)), [-1], [1E-5])))
    wave = wave[:, :n1]  # get rid of padding before returning

    return wave, period, scale, coi


# --------------------------------------------------------------------------
# WAVE_BASES  1D Wavelet functions Morlet, Paul, or DOG
#
#  DAUGHTER,FOURIER_FACTOR,COI,DOFMIN = wave_bases(MOTHER,K,SCALE,PARAM)
#
#   Computes the wavelet function as a function of Fourier frequency,
#   used for the wavelet transform in Fourier space.
#   (This program is called automatically by WAVELET)
#
# INPUTS:
#
#    MOTHER = a string, equal to 'MORLET' or 'PAUL' or 'DOG'
#    K = a vector, the Fourier frequencies at which to calculate the wavelet
#    SCALE = a number, the wavelet scale
#    PARAM = the nondimensional parameter for the wavelet function
#
# OUTPUTS:
#
#    DAUGHTER = a vector, the wavelet function
#    FOURIER_FACTOR = the ratio of Fourier period to scale
#    COI = a number, the cone-of-influence size at the scale
#    DOFMIN = a number, degrees of freedom for each point in the wavelet power
#             (either 2 for Morlet and Paul, or 1 for the DOG)

def wave_bases(mother, k, scale, param):
    n = len(k)
    kplus = np.array(k > 0., dtype=float)

    if mother == 'MORLET':  # -----------------------------------  Morlet

        if param == -1:
            param = 6.

        k0 = np.copy(param)
        # calc psi_0(s omega) from Table 1
        expnt = -(scale * k - k0) ** 2 / 2. * kplus
        norm = np.sqrt(scale * k[1]) * (np.pi ** (-0.25)) * np.sqrt(n)
        daughter = norm * np.exp(expnt)
        daughter = daughter * kplus  # Heaviside step function
        # Scale-->Fourier [Sec.3h]
        fourier_factor = (4 * np.pi) / (k0 + np.sqrt(2 + k0 ** 2))
        coi = fourier_factor / np.sqrt(2)  # Cone-of-influence [Sec.3g]
        dofmin = 2  # Degrees of freedom
    elif mother == 'PAUL':  # --------------------------------  Paul
        if param == -1:
            param = 4.
        m = param
        # calc psi_0(s omega) from Table 1
        expnt = -scale * k * kplus
        norm_bottom = np.sqrt(m * np.prod(np.arange(1, (2 * m))))
        norm = np.sqrt(scale * k[1]) * (2 ** m / norm_bottom) * np.sqrt(n)
        daughter = norm * ((scale * k) ** m) * np.exp(expnt) * kplus
        fourier_factor = 4 * np.pi / (2 * m + 1)
        coi = fourier_factor * np.sqrt(2)
        dofmin = 2
    elif mother == 'DOG':  # --------------------------------  DOG
        if param == -1:
            param = 2.
        m = param
        # calc psi_0(s omega) from Table 1
        expnt = -(scale * k) ** 2 / 2.0
        norm = np.sqrt(scale * k[1] / gamma(m + 0.5)) * np.sqrt(n)
        daughter = -norm * (1j ** m) * ((scale * k) ** m) * np.exp(expnt)
        fourier_factor = 2 * np.pi * np.sqrt(2. / (2 * m + 1))
        coi = fourier_factor / np.sqrt(2)
        dofmin = 1
    else:
        print('Mother must be one of MORLET, PAUL, DOG')

    return daughter, fourier_factor, coi, dofmin


# --------------------------------------------------------------------------
# WAVE_SIGNIF  Significance testing for the 1D Wavelet transform WAVELET
#
#   SIGNIF = wave_signif(Y,DT,SCALE,SIGTEST,LAG1,SIGLVL,DOF,MOTHER,PARAM)
#
# INPUTS:
#
#    Y = the time series, or, the VARIANCE of the time series.
#        (If this is a single number, it is assumed to be the variance...)
#    DT = amount of time between each Y value, i.e. the sampling time.
#    SCALE = the vector of scale indices, from previous call to WAVELET.
#
#
# OUTPUTS:
#
#    SIGNIF = significance levels as a function of SCALE
#    FFT_THEOR = output theoretical red-noise spectrum as fn of PERIOD
#
#
# OPTIONAL INPUTS:
#    SIGTEST = 0, 1, or 2.    If omitted, then assume 0.
#
#         If 0 (the default), then just do a regular chi-square test,
#             i.e. Eqn (18) from Torrence & Compo.
#         If 1, then do a "time-average" test, i.e. Eqn (23).
#             In this case, DOF should be set to NA, the number
#             of local wavelet spectra that were averaged together.
#             For the Global Wavelet Spectrum, this would be NA=N,
#             where N is the number of points in your time series.
#         If 2, then do a "scale-average" test, i.e. Eqns (25)-(28).
#             In this case, DOF should be set to a
#             two-element vector [S1,S2], which gives the scale
#             range that was averaged together.
#             e.g. if one scale-averaged scales between 2 and 8,
#             then DOF=[2,8].
#
#    LAG1 = LAG 1 Autocorrelation, used for SIGNIF levels. Default is 0.0
#
#    SIGLVL = significance level to use. Default is 0.95
#
#    DOF = degrees-of-freedom for signif test.
#         IF SIGTEST=0, then (automatically) DOF = 2 (or 1 for MOTHER='DOG')
#         IF SIGTEST=1, then DOF = NA, the number of times averaged together.
#         IF SIGTEST=2, then DOF = [S1,S2], the range of scales averaged.
#
#       Note: IF SIGTEST=1, then DOF can be a vector (same length as SCALEs),
#            in which case NA is assumed to vary with SCALE.
#            This allows one to average different numbers of times
#            together at different scales, or to take into account
#            things like the Cone of Influence.
#            See discussion following Eqn (23) in Torrence & Compo.
#
#    GWS = global wavelet spectrum, a vector of the same length as scale.
#          If input then this is used as the theoretical background spectrum,
#          rather than white or red noise.

def wave_signif(Y, dt, scale, sigtest=0, lag1=0.0, siglvl=0.95,
                dof=None, mother='MORLET', param=None, gws=None):
    n1 = len(np.atleast_1d(Y))
    J1 = len(scale) - 1
    dj = np.log2(scale[1] / scale[0])

    if n1 == 1:
        variance = Y
    else:
        variance = np.std(Y) ** 2

    # get the appropriate parameters [see Table(2)]
    if mother == 'MORLET':  # ----------------------------------  Morlet
        empir = ([2., -1, -1, -1])
        if param is None:
            param = 6.
            empir[1:] = ([0.776, 2.32, 0.60])
        k0 = param
        # Scale-->Fourier [Sec.3h]
        fourier_factor = (4 * np.pi) / (k0 + np.sqrt(2 + k0 ** 2))
    elif mother == 'PAUL':
        empir = ([2, -1, -1, -1])
        if param is None:
            param = 4
            empir[1:] = ([1.132, 1.17, 1.5])
        m = param
        fourier_factor = (4 * np.pi) / (2 * m + 1)
    elif mother == 'DOG':  # -------------------------------------Paul
        empir = ([1., -1, -1, -1])
        if param is None:
            param = 2.
            empir[1:] = ([3.541, 1.43, 1.4])
        elif param == 6:  # --------------------------------------DOG
            empir[1:] = ([1.966, 1.37, 0.97])
        m = param
        fourier_factor = 2 * np.pi * np.sqrt(2. / (2 * m + 1))
    else:
        print('Mother must be one of MORLET, PAUL, DOG')

    period = scale * fourier_factor
    dofmin = empir[0]  # Degrees of freedom with no smoothing
    Cdelta = empir[1]  # reconstruction factor
    gamma_fac = empir[2]  # time-decorrelation factor
    dj0 = empir[3]  # scale-decorrelation factor

    freq = dt / period  # normalized frequency

    if gws is not None:   # use global-wavelet as background spectrum
        fft_theor = gws
    else:
        # [Eqn(16)]
        fft_theor = (1 - lag1 ** 2) / \
            (1 - 2 * lag1 * np.cos(freq * 2 * np.pi) + lag1 ** 2)
        fft_theor = variance * fft_theor  # include time-series variance

    signif = fft_theor
    if dof is None:
        dof = dofmin

    if sigtest == 0:  # no smoothing, DOF=dofmin [Sec.4]
        dof = dofmin
        chisquare = chisquare_inv(siglvl, dof) / dof
        signif = fft_theor * chisquare  # [Eqn(18)]
    elif sigtest == 1:  # time-averaged significance
        if len(np.atleast_1d(dof)) == 1:
            dof = np.zeros(J1) + dof
        dof[dof < 1] = 1
        # [Eqn(23)]
        dof = dofmin * np.sqrt(1 + (dof * dt / gamma_fac / scale) ** 2)
        dof[dof < dofmin] = dofmin   # minimum DOF is dofmin
        for a1 in range(0, J1 + 1):
            chisquare = chisquare_inv(siglvl, dof[a1]) / dof[a1]
            signif[a1] = fft_theor[a1] * chisquare
    elif sigtest == 2:  # time-averaged significance
        if len(dof) != 2:
            print('ERROR: DOF must be set to [S1,S2],'
                ' the range of scale-averages')
        if Cdelta == -1:
            print('ERROR: Cdelta & dj0 not defined'
                  ' for ' + mother + ' with param = ' + str(param))

        s1 = dof[0]
        s2 = dof[1]
        avg = np.logical_and(scale >= 2, scale < 8)  # scales between S1 & S2
        navg = np.sum(np.array(np.logical_and(scale >= 2, scale < 8),
            dtype=int))
        if navg == 0:
            print('ERROR: No valid scales between ' + s1 + ' and ' + s2)
        Savg = 1. / np.sum(1. / scale[avg])  # [Eqn(25)]
        Smid = np.exp((np.log(s1) + np.log(s2)) / 2.)  # power-of-two midpoint
        dof = (dofmin * navg * Savg / Smid) * \
            np.sqrt(1 + (navg * dj / dj0) ** 2)  # [Eqn(28)]
        fft_theor = Savg * np.sum(fft_theor[avg] / scale[avg])  # [Eqn(27)]
        chisquare = chisquare_inv(siglvl, dof) / dof
        signif = (dj * dt / Cdelta / Savg) * fft_theor * chisquare  # [Eqn(26)]
    else:
        print('ERROR: sigtest must be either 0, 1, or 2')

    return signif


# --------------------------------------------------------------------------
# CHISQUARE_INV  Inverse of chi-square cumulative distribution function (cdf).
#
#   X = chisquare_inv(P,V) returns the inverse of chi-square cdf with V
#   degrees of freedom at fraction P.
#   This means that P*100 percent of the distribution lies between 0 and X.
#
#   To check, the answer should satisfy:   P==gammainc(X/2,V/2)

# Uses FMIN and CHISQUARE_SOLVE

def chisquare_inv(P, V):

    if (1 - P) < 1E-4:
        print('P must be < 0.9999')

    if P == 0.95 and V == 2:  # this is a no-brainer
        X = 5.9915
        return X

    MINN = 0.01  # hopefully this is small enough
    MAXX = 1  # actually starts at 10 (see while loop below)
    X = 1
    TOLERANCE = 1E-4  # this should be accurate enough

    while (X + TOLERANCE) >= MAXX:  # should only need to loop thru once
        MAXX = MAXX * 10.
    # this calculates value for X, NORMALIZED by V
        X = fminbound(chisquare_solve, MINN, MAXX, args=(P, V), xtol=TOLERANCE)
        MINN = MAXX

    X = X * V  # put back in the goofy V factor

    return X  # end of code


# --------------------------------------------------------------------------
# CHISQUARE_SOLVE  Internal function used by CHISQUARE_INV
    #
    #   PDIFF=chisquare_solve(XGUESS,P,V)  Given XGUESS, a percentile P,
    #   and degrees-of-freedom V, return the difference between
    #   calculated percentile and P.

    # Uses GAMMAINC
    #
    # Written January 1998 by C. Torrence

    # extra factor of V is necessary because X is Normalized

def chisquare_solve(XGUESS, P, V):

    PGUESS = gammainc(V / 2, V * XGUESS / 2)  # incomplete Gamma function

    PDIFF = np.abs(PGUESS - P)            # error in calculated P

    TOL = 1E-4
    if PGUESS >= 1 - TOL:  # if P is very close to 1 (i.e. a bad guess)
        PDIFF = XGUESS   # then just assign some big number like XGUESS

    return PDIFF

In [6]:
def cwt(sst, cutoff_period=3):
    sst = sst - np.mean(sst)
    variance = np.std(sst, ddof=1) ** 2
    if 0:
        variance = 1.0
        sst = sst / np.std(sst, ddof=1)
    n = len(sst)
    dt = 1
    time = np.arange(len(sst)) * dt  # construct time array
    xlim = ([0, 300])  # plotting range
    pad = 1  # pad the time series with zeroes (recommended)
    dj = 0.25  # this will do 4 sub-octaves per octave
    s0 = 2 * dt  # this says start at a scale of 6 months
    j1 = 7 / dj  # this says do 7 powers-of-two with dj sub-octaves each
    lag1 = 0.72  # lag-1 autocorrelation for red noise background
    mother = 'MORLET'

    # Wavelet transform:
    wave, period, scale, coi = wavelet(sst, dt, pad, dj, s0, j1, mother)
    power = (np.abs(wave)) ** 2  # compute wavelet power spectrum
    scale_avg = scale[:, np.newaxis].dot(np.ones(n)[np.newaxis, :])
    power = power / scale_avg  # [Eqn(24)]
    # REMOVE SMALL PERIODS and those under cone of influence
    k = 0
    while period[k] < cutoff_period:
        k += 1
    for i in range(k):
        for j in range(len(power[0])):
            power[i, j] = 0
    for i in range(len(coi)):
        j = 0
        while period[j] <= coi[i]:
            j += 1
        while j < len(period):
            power[j, i] = 0
            j += 1
    return power

<a id='gen'></a>
___
### Генерация 40000 синтетических сигналов

[Вернуться наверх](#up)

In [ ]:
def add_qpp(x, A_flare, l_flare):
    phi = np.random.uniform(0, 2 * np.pi)
    select_P = [10, 20, 30]
    select_A_qpp = [0.1, 0.2, 0.3]
    select_t_e = [1/10, 2/15, 1/5, 2/5]
    P = l_flare / random.choice(select_P)
    A_qpp = A_flare * random.choice(select_A_qpp)
    t_e = l_flare * random.choice(select_t_e)
    return A_qpp * np.exp(-x / t_e) * np.cos(2 * np.pi * x / P + phi)

def add_noise(S):
    N_white = S / random.choice(np.arange(3, 6))
    r = np.random.uniform(0.81, 0.99)
    N_red = S / max(1, 17 + np.random.normal(0, 1))
    white_noise = np.random.normal(0, N_white, 300)
    red_noise = np.zeros(300)
    red_noise[0] = N_red / 2
    for i in range(1, 300):
        red_noise[i] = r * red_noise[i - 1] + \
        np.power((1 - r * r), 0.5) * white_noise[i]
    noise = np.add(white_noise, red_noise)
    return noise

def polynomial(x, A=1):
    x0 = 1
    x1 = 1.941 + np.random.uniform(-0.008, 0.008)
    x2 = -0.175 + np.random.uniform(-0.032, 0.032)
    x3 = -2.246 + np.random.uniform(-0.039, 0.039)
    x4 = -1.125 + np.random.uniform(-0.016, 0.016)
    return A * (x0 + x1 * x + x2 * np.power(x, 2.) + x3 * np.power(x, 3.) + \
x4 * np.power(x, 4.))

def exp1(x, A=1):
    return A * 0.948 * np.exp(-0.965 * x)

def exp2(x, A=1):
    return A * 0.322 * np.exp(-0.290 * x)

def generate_exponential(contains_qpp=False):
    l_flare = np.random.uniform(100, 200)
    t_peak = np.random.randint(30, 300 - l_flare)
    A_flare = 10 + np.random.normal(0, 4)
    if A_flare < 0.5:
        A_flare = 0.5
    peak_flare = l_flare * t_peak / 300
    points1 = np.linspace(-1.0, 0.0, t_peak - 1, endpoint=False)
    num_point2 = int((300 - t_peak + 1) * 1.6 / 7)
    points2 = np.linspace(0.0, 1.6, num_point2 - 1, endpoint=False)
    points3 = np.linspace(1.6, 7, 300 - t_peak + 1 - num_point2 + 1)
    points_qpp = np.linspace(0.0, l_flare - peak_flare, 300 - t_peak + 1)
    trend1 = polynomial(points1, A_flare)
    trend2 = np.concatenate([exp1(points2, A_flare), exp2(points3, A_flare)])
    if contains_qpp:
        trend = np.concatenate([trend1, \
                                np.add(trend2, add_qpp(points_qpp, A_flare, l_flare))])
    else:
        trend = np.concatenate([trend1, trend2])
    trend_with_noise = np.add(trend, add_noise(np.mean(trend)))
    return trend_with_noise

exponential_no_qpp = 0
exponential_qpp = 0

with open('one_type_signals.txt', 'w') as f:
    for i in range(40000):
        has_qpp = np.random.randint(2)
        inf = np.zeros(2)
        inf[0] = has_qpp
        inf[1] = flare_type
        if has_qpp:
            flare = generate_exponential(True)
            res = np.concatenate([inf, flare])
            f.write("%s\n" % ' '.join(list(map(str, res))))
            exponential_qpp += 1
        else:
            flare = generate_exponential(False)
            res = np.concatenate([inf, flare])
            f.write("%s\n" % ' '.join(list(map(str, res))))
            exponential_no_qpp += 1

<a id='info'></a>
___
### Функция для получения информации об областях значимости

[Вернуться наверх](#up)

In [10]:
def running_smoothing(sign):
    return sign - pd.Series(sign).rolling(10, min_periods=1).mean()

def center_signal(sign):
    m = np.mean(sign)
    for j in range(0, len(sign)):
        sign[j] -= m
    return sign

def change_signal(sign):
    ind = np.argmax(sign)
    sign2 = sign[ind:]
    detrended = running_smoothing(sign2)
    sign0 = np.hstack((np.zeros(300-len(detrended)), detrended))
    return center_signal(sign0)

def find_info(signal):
    info = np.zeros(6)
    
    sst = signal
    sst = change_signal(sst)
    sst = sst - np.mean(sst)
    variance = np.std(sst, ddof=1) ** 2
    if 0:
        variance = 1.0
        sst = sst / np.std(sst, ddof=1)
    n = len(sst)
    dt = 1
    time = np.arange(len(sst)) * dt
    xlim = ([0, 300])
    pad = 1
    dj = 0.25
    s0 = 2 * dt
    j1 = 7 / dj
    lag1 = 0.72
    mother = 'MORLET'

    wave, period, scale, coi = wavelet(sst, dt, pad, dj, s0, j1, mother)
    power = (np.abs(wave)) ** 2
    global_ws = (np.sum(power, axis=1) / n)

    k = 0
    while period[k] < 3:
        k += 1
    for i in range(3):
        for j in range(len(power[0])):
            power[i, j] = 0
    for i in range(len(coi)):
        j = 0
        while period[j] <= coi[i]:
            j += 1
        while j < len(period):
            power[j, i] = 0
            j += 1
    signif = wave_signif(([variance]), dt=dt, sigtest=0, scale=scale,
        lag1=lag1, siglvl=0.95, mother=mother)
    sig99 = signif[:, np.newaxis].dot(np.ones(n)[np.newaxis, :])
    sig99 = power / sig99
    
    dof = n - scale
    global_signif = wave_signif(variance, dt=dt, scale=scale, sigtest=1,
        lag1=lag1, dof=dof, mother=mother)
    CN = plt.contour(time, period, sig99, [-99, 1], colors='k')
    figures = []
    max_area = 0
    if len(CN.allsegs) <= 1:
        return info
    for ii, seg in enumerate(CN.allsegs[1]):
        pgon = Polygon(zip(seg[:,0], seg[:,1]))
        bound = pgon.bounds # bounds: (minx, miny, maxx, maxy)
        area = pgon.area
        width = bound[2] - bound[0]
        height = bound[3] - bound[1]
        center_x = pgon.centroid.x
        center_y = pgon.centroid.y
        if area > max_area and center_y > 5 and center_y < 50:
            max_area = area
            info[0] = area
            info[1] = width / center_y
            info[2] = width
            info[3] = height
            info[4] = center_x
            info[5] = center_y
    return info

In [110]:
%matplotlib agg
%matplotlib agg

**Получение информации об облястях значимости.**

In [ ]:
len_file = 0
with open('one_type_signals_info.txt', 'w') as f:
    for i in range(len(X)):
        has_qpp = np.zeros(1)
        has_qpp[0] = y[i]
        info = find_info(X[i])
        res = np.concatenate([has_qpp, info])
        f.write("%s\n" % ' '.join(list(map(str, res))))
        len_file += 1

In [ ]:
data = np.loadtxt('one_type_signals_info.txt')
y = data[:,0]
X = data[:,2:]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
print(len(X_train), X_train[0].size)

<a id='network1'></a>
___
### Перцептрон 

[Вернуться наверх](#up)

In [115]:
model = Sequential()
model.add(Dense(12, input_dim=5, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', \
                                            patience=10, verbose=0, mode='auto')
model.fit(X_train, y_train, epochs=12, batch_size=32, verbose=1, \
          validation_data=(X_test, y_test), callbacks=[callback_early_stopping])
# evaluate the keras model
train_score = model.evaluate(X_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Epoch 1/12
1000/1000 [==============================] - 1s 832us/step - loss: 0.8236 - accuracy: 0.7446 - val_loss: 0.3933 - val_accuracy: 0.8319
Epoch 2/12
1000/1000 [==============================] - 1s 640us/step - loss: 0.4072 - accuracy: 0.8266 - val_loss: 0.4128 - val_accuracy: 0.8231
Epoch 3/12
1000/1000 [==============================] - 1s 737us/step - loss: 0.4015 - accuracy: 0.8282 - val_loss: 0.4213 - val_accuracy: 0.8250
Epoch 4/12
1000/1000 [==============================] - 1s 647us/step - loss: 0.4019 - accuracy: 0.8281 - val_loss: 0.4055 - val_accuracy: 0.8264
Epoch 5/12
1000/1000 [==============================] - 1s 633us/step - loss: 0.4037 - accuracy: 0.8258 - val_loss: 0.4095 - val_accuracy: 0.8229
Epoch 6/12
1000/1000 [==============================] - 1s 733us/step - loss: 0.3995 - accuracy: 0.8288 - val_loss: 0.4012 - val_accuracy: 0.8340
Epoch 7/12
1000/1000 [==============================] - 1s 679us/step - loss: 0.4014 - accuracy: 0.8296 - val_loss: 0.3925 -

<a id='network2'></a>
___
### Нейронная сеть с двумя входами

[Вернуться наверх](#up)

In [116]:
data1 = np.loadtxt('one_type_signals_info.txt')
y1 = data1[:,0]
X1 = data1[:,2:]
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X1, y1, test_size=0.2, random_state=42)
print(len(X_train1), X_train1[0].size)

data2 = np.loadtxt('one_type_signals.txt')
y2 = data2[:,0]
X2 = data2[:,2:]
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X2, y2, test_size=0.2, random_state=42)
print(len(X_train2), X_train2[0].size)

32000 5
32000 300


In [122]:
def running_smoothing(sign):
    detrended = sign - pd.Series(sign).rolling(10, min_periods=1).mean()
    return detrended

def center_signal(sign):
    m = np.mean(sign)
    for j in range(0, len(sign)):
        sign[j] -= m
    return sign

def change_signal(sign):
    ind = np.argmax(sign)
    sign2 = sign[ind:]
    detrended = running_smoothing(sign2)
    sign0 = np.hstack((np.zeros(300-len(detrended)), detrended))
    return center_signal(sign0)

train_size2 = 32000
test_size2 = 8000
train_data_cwt = np.ndarray(shape=(train_size2, 29, 300))
for i in range(0, train_size2):
    signal = X_train2[i]
    signal = change_signal(signal)
    coeff_ = cwt(signal)
    train_data_cwt[i, :, :] = coeff_

test_data_cwt = np.ndarray(shape=(test_size2, 29, 300))
for i in range(0,test_size2):
    signal = X_test2[i]
    signal = change_signal(signal)
    coeff_ = cwt(signal)
    test_data_cwt[i, :, :] = coeff_
    
history = History()

In [123]:
x_train2 = train_data_cwt
x_test2 = test_data_cwt
img_x = 29
img_y = 300
num_classes = 2

batch_size = 32
epochs = 10

input_shape = (img_x, img_y)

x_train2 = x_train2.astype('float32')
x_test2 = x_test2.astype('float32')

print('x_train shape:', x_train2.shape)
print(x_train2.shape[0], 'train samples')
print(x_test2.shape[0], 'test samples')

x_train shape: (32000, 29, 300)
32000 train samples
8000 test samples


In [127]:
from keras.layers import concatenate
from keras.models import Model


cnn = Sequential()
cnn.add(Conv1D(32, kernel_size=5, strides=1, activation='relu', input_shape=input_shape)) 
cnn.add(MaxPooling1D(pool_size=2, strides=2))
cnn.add(Conv1D(64, 5, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

mlp = Sequential()
mlp.add(Dense(12, input_dim=5, activation='relu'))
mlp.add(Dense(8, activation='relu'))
mlp.add(Dense(1, activation='sigmoid'))

combinedInput = concatenate([mlp.output, cnn.output])
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# train the model
model.fit(x=[X_train1, x_train2], y=y_train1,\
          validation_data=([X_test1, x_test2], y_test1),\
          epochs=15, batch_size=32, callbacks=[history])

train_score = model.evaluate(x=[X_train1, x_train2], y=y_train1, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate([X_test1, x_test2], y_test1, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Epoch 1/15
1000/1000 [==============================] - 10s 9ms/step - loss: 0.4350 - accuracy: 0.7868 - val_loss: 0.2960 - val_accuracy: 0.8790
Epoch 2/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3051 - accuracy: 0.8759 - val_loss: 0.3015 - val_accuracy: 0.8846
Epoch 3/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2911 - accuracy: 0.8893 - val_loss: 0.3038 - val_accuracy: 0.8829
Epoch 4/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2784 - accuracy: 0.8904 - val_loss: 0.2741 - val_accuracy: 0.8874
Epoch 5/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2663 - accuracy: 0.8962 - val_loss: 0.2878 - val_accuracy: 0.8865
Epoch 6/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2570 - accuracy: 0.8979 - val_loss: 0.2765 - val_accuracy: 0.8852
Epoch 7/15
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2523 - accuracy: 0.8993 - val_loss: 0.2885 - val_accuracy

<a id='network3'></a>
___
### Сверточная нейронная сеть

[Вернуться наверх](#up)

In [4]:
data = np.loadtxt('one_type_signals.txt')
y = data[:,0]
X = data[:,2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), X_train[0].size)

32000 300


In [7]:
def running_smoothing(sign):
    detrended = sign - pd.Series(sign).rolling(10, min_periods=0, center=True).mean()
    return detrended

def center_signal(sign):
    m = np.mean(sign)
    for j in range(0, len(sign)):
        sign[j] -= m
    return sign

def change_signal(sign):
    ind = np.argmax(sign)
    sign2 = sign[ind:]
    detrended = running_smoothing(sign2)
    sign0 = np.hstack((np.zeros(300-len(detrended)), detrended))
    return center_signal(sign0)

train_size = 32000
test_size = 8000

train_data_cwt = np.ndarray(shape=(train_size, 29, 300))
for i in range(0, train_size):
    signal = X_train[i]
    signal = change_signal(signal)
    coeff_ = cwt(signal)
    train_data_cwt[i, :, :] = coeff_

test_data_cwt = np.ndarray(shape=(test_size, 29, 300))
for i in range(0,test_size):
    signal = X_test[i]
    signal = change_signal(signal)
    coeff_ = cwt(signal)
    test_data_cwt[i, :, :] = coeff_
    
history20 = History()

In [8]:
x_train = train_data_cwt
x_test = test_data_cwt
img_x = 29
img_y = 300
num_classes = 2

batch_size = 32
epochs = 10

input_shape = (img_x, img_y)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (32000, 29, 300)
32000 train samples
8000 test samples


In [9]:
model20 = Sequential()
model20.add(Conv1D(32, kernel_size=5, strides=1, activation='relu', input_shape=input_shape)) 
model20.add(MaxPooling1D(pool_size=2, strides=2))
model20.add(Conv1D(64, 5, activation='relu'))
model20.add(MaxPooling1D(pool_size=2))
model20.add(Flatten())
model20.add(Dense(1024, activation='relu'))
model20.add(Dense(num_classes, activation='softmax'))

model20.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer="adam", 
              metrics=['accuracy'])

model20.fit(x_train, y_train, batch_size=batch_size, 
          epochs=epochs, verbose=1, 
          validation_data=(x_test, y_test), 
          callbacks=[history20])

train_score = model20.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model20.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Epoch 1/10
1000/1000 [==============================] - 29s 10ms/step - loss: 0.3955 - accuracy: 0.8195 - val_loss: 0.3024 - val_accuracy: 0.8698
Epoch 2/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2969 - accuracy: 0.8768 - val_loss: 0.2915 - val_accuracy: 0.8783
Epoch 3/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2861 - accuracy: 0.8804 - val_loss: 0.3022 - val_accuracy: 0.8664
Epoch 4/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2768 - accuracy: 0.8854 - val_loss: 0.3013 - val_accuracy: 0.8715
Epoch 5/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2703 - accuracy: 0.8902 - val_loss: 0.2894 - val_accuracy: 0.8766
Epoch 6/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2611 - accuracy: 0.8907 - val_loss: 0.2840 - val_accuracy: 0.8827
Epoch 7/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2551 - accuracy: 0.8938 - val_loss: 0.2978 - val_accurac